In [2]:
pip install transformers datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
from datasets import load_dataset

# Load CoNLL-2003 dataset
dataset = load_dataset("conll2003")
print(dataset["train"].features)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/12.3k [00:00<?, ?B/s]

conll2003.py:   0%|          | 0.00/9.57k [00:00<?, ?B/s]

The repository for conll2003 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/conll2003.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

{'id': Value(dtype='string', id=None), 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'pos_tags': Sequence(feature=ClassLabel(names=['"', "''", '#', '$', '(', ')', ',', '.', ':', '``', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'NN|SYM', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB'], id=None), length=-1, id=None), 'chunk_tags': Sequence(feature=ClassLabel(names=['O', 'B-ADJP', 'I-ADJP', 'B-ADVP', 'I-ADVP', 'B-CONJP', 'I-CONJP', 'B-INTJ', 'I-INTJ', 'B-LST', 'I-LST', 'B-NP', 'I-NP', 'B-PP', 'I-PP', 'B-PRT', 'I-PRT', 'B-SBAR', 'I-SBAR', 'B-UCP', 'I-UCP', 'B-VP', 'I-VP'], id=None), length=-1, id=None), 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)}


In [10]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'ner_tags_label', 'entity_pairs'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'ner_tags_label', 'entity_pairs'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'ner_tags_label', 'entity_pairs'],
        num_rows: 3453
    })
})

In [ ]:
dataset['train'].features['ner_tags'].feature.names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [ ]:
dataset['train'][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [4]:
# Extract the label names from the dataset's 'ner_tags' feature
label_list = dataset["train"].features["ner_tags"].feature.names

# Map integer NER tags to string labels
def convert_tags_to_labels(example):
    example["ner_tags_label"] = [label_list[tag] for tag in example["ner_tags"]]
    return example

# Apply the conversion to each split
dataset = dataset.map(convert_tags_to_labels)


Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'ner_tags_label'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'ner_tags_label'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'ner_tags_label'],
        num_rows: 3453
    })
})

In [44]:
from collections import defaultdict

# Define a basic rule-based approach for relationships between entities
relationship_labels = ["ORG-LOC", "PER-ORG", "PER-PER", "ORG-ORG", "NO_RELATION"]

relationship_labels = ["ORG-LOC", "PER-ORG", "PER-PER", "ORG-ORG", "NO_RELATION"]

def extract_entity_pairs_and_relations(example):
    tokens = example["tokens"]
    ner_tags = example["ner_tags_label"]

    entities = []
    current_entity = ""
    current_label = ""
    entity_start = -1

    # Extract entities from tokens and NER tags
    for i, (token, tag) in enumerate(zip(tokens, ner_tags)):
        if tag.startswith("B-"):
            if current_entity:
                entities.append((current_entity, current_label, entity_start, i - 1))
            current_entity = token
            current_label = tag[2:]
            entity_start = i
        elif tag.startswith("I-") and current_label:
            current_entity += " " + token
        else:
            if current_entity:
                entities.append((current_entity, current_label, entity_start, i - 1))
            current_entity = ""
            current_label = ""

    if current_entity:
        entities.append((current_entity, current_label, entity_start, len(tokens) - 1))

    # Generate entity pairs with labels
    pairs = []
    for i, (entity1, label1, start1, end1) in enumerate(entities):
        for j, (entity2, label2, start2, end2) in enumerate(entities):
            if i < j:  # Avoid duplicate pairs
                if label1 == "ORG" and label2 == "LOC":
                    pairs.append((entity1, entity2, "ORG-LOC"))
                elif label1 == "PER" and label2 == "ORG":
                    pairs.append((entity1, entity2, "PER-ORG"))
                elif label1 == "PER" and label2 == "PER":
                    pairs.append((entity1, entity2, "PER-PER"))
                elif label1 == "ORG" and label2 == "ORG":
                    pairs.append((entity1, entity2, "ORG-ORG"))
                else:
                    pairs.append((entity1, entity2, "NO_RELATION"))

    if not pairs:  # If no entity pairs were found
        pairs.append(("", "", "NO_RELATION"))  # Add a default "NO_RELATION"

    return pairs

# Apply extraction to the dataset
dataset = dataset.map(lambda example: {"entity_pairs": extract_entity_pairs_and_relations(example)})


Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [ ]:
dataset['train']['entity_pairs']


In [15]:
df = dataset["train"].select(range(100))

In [16]:
df

Dataset({
    features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'ner_tags_label', 'entity_pairs'],
    num_rows: 100
})

In [45]:
def add_entity_markers(example):
    sentences = []
    relations = []

    # Flatten tokens if needed (to ensure tokens is a list of strings)
    tokens = example.get("tokens", [])
    sentence_str = " ".join(tokens) if tokens else ""

    # Check if entity_pairs exist and are non-empty
    entity_pairs = example.get("entity_pairs", [])
    if not entity_pairs:
        # If entity_pairs is empty, add the original sentence without markers
        sentences.append(sentence_str)
        relations.append(None)  # or an empty string, if preferred
        return {"marked_sentences": sentences, "relations": relations}

    # Process each entity pair if entity_pairs is not empty
    for entity_pair in entity_pairs:
        # Ensure the entity_pair has exactly three elements
        if len(entity_pair) != 3:
            print(f"Skipping invalid entity_pair: {entity_pair}")
            continue

        # Unpack the entity pair, assuming each entity_pair is [entity1, entity2, relation]
        entity1, entity2, relation = entity_pair

        # Ensure entity1 and entity2 are strings (join if they are lists of tokens)
        entity1 = " ".join(entity1) if isinstance(entity1, list) else entity1
        entity2 = " ".join(entity2) if isinstance(entity2, list) else entity2

        # Add markers around the entities in the sentence
        sentence_with_markers = sentence_str.replace(entity1, f"[E1]{entity1}[/E1]").replace(entity2, f"[E2]{entity2}[/E2]")

        # Append the modified sentence and corresponding relation
        sentences.append(sentence_with_markers)
        relations.append(relation)

    # Return lists of marked sentences and relations for this example
    return {"marked_sentences": sentences, "relations": relations}

# Apply function to dataset without batching
data = dataset.map(add_entity_markers)


Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [ ]:
data['train']['marked_sentences']

In [96]:
dataset['train'][0]['entity_pairs']

[['EU', 'German', 'NO_RELATION'],
 ['EU', 'British', 'NO_RELATION'],
 ['German', 'British', 'NO_RELATION']]

In [12]:
relationship_labels

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [46]:
from transformers import RobertaTokenizerFast

# Initialize the tokenizer for RoBERTa
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")

relationship_labels
label2id = {label: i for i, label in enumerate(relationship_labels)}
id2label = {i: label for label, i in label2id.items()}

def tokenize_function(examples):
    # Ensure all inputs are strings, replacing None with empty strings
    marked_sentences = [str(sentence) if sentence is not None else "" for sentence in examples["marked_sentences"]]

    # Tokenize the batch of sentences
    encodings = tokenizer.batch_encode_plus(
        marked_sentences,
        truncation=True,
        padding="max_length",
        max_length=128,
        return_tensors="tf"
    )

    # Replace None or empty lists in relations with ["NO_RELATION"]
    relations = examples["relations"] if examples["relations"] is not None else [["NO_RELATION"]]
    relations = [rel if rel else ["NO_RELATION"] for rel in relations]  # Handle empty lists

    # Map each relation label in the batch to its corresponding ID
    encodings["labels"] = [[label2id.get(relation, label2id["NO_RELATION"]) for relation in relation_list] for relation_list in relations]

    return encodings

# Apply tokenization to the dataset
#tokenized_dataset = data["train"].map(tokenize_function, batched=True)



# Tokenize dataset
tokenized_data = data.map(tokenize_function, batched=True)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [154]:
for batch in tokenized_data["train"].take(3):
    print("Labels:", batch["labels"])

Labels: [4, 4, 4]
Labels: [4]
Labels: [4]


In [140]:
''' def tokenize_function_al(examples):
    # Ensure all inputs are strings, replacing None with empty strings
    marked_sentences = [str(sentence) if sentence is not None else "" for sentence in examples["marked_sentences"]]

    # Tokenize the batch of sentences
    encodings = tokenizer(
        marked_sentences,
        truncation=True,
        padding="max_length",
        max_length=128
    )

    # Process each relation as a single sequence-level label
    relations = examples["relations"]
    relations = [relation if relation else "NO_RELATION" for relation in relations]  # Handle empty relations

    # Map each relation label to its corresponding ID
    encodings["labels"] = [label2id.get(relation, label2id["NO_RELATION"]) for relation in relations]

    return encodings '''



In [130]:
''' def tokenize_function_ex(examples):
    # Ensure all inputs are strings, replacing None with empty strings
    marked_sentences = [str(sentence) if sentence is not None else "" for sentence in examples["marked_sentences"]]

    # Tokenize the batch of sentences with truncation and padding
    encodings = tokenizer(
        marked_sentences,
        truncation=True,
        padding="max_length",
        max_length=128
    )

    # Flatten relations to ensure they are single strings
    relations = examples["relations"]
    # If any relation is a list, take the first element as the main label
    relations = [relation[0] if isinstance(relation, list) else relation for relation in relations]
    relations = [relation if relation else "NO_RELATION" for relation in relations]  # Handle empty relations

    # Map each relation label to its corresponding ID
    encodings["labels"] = [label2id.get(relation, label2id["NO_RELATION"]) for relation in relations]

    return {
        "input_ids": encodings["input_ids"],
        "attention_mask": encodings["attention_mask"],
        "labels": encodings["labels"],  # Ensure labels are a single value per example
    }

# Apply the function to the dataset and ensure batched processing
tokenized_datas = data.map(tokenize_function_ex, batched=True) '''




Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [139]:
''' max_length = 128
def pad_labels(labels, max_length, pad_value=-100):
    padded_labels = []
    for seq in labels:
        # Flatten if needed and ensure each is a list
        if isinstance(seq, list):
            seq = seq[:max_length] + [pad_value] * (max_length - len(seq))
        else:
            # Convert single values to lists and pad
            seq = [seq] + [pad_value] * (max_length - 1)
        padded_labels.append(seq)
    return padded_labels



# Apply padding after tokenizing
#tokenized_dataset = data["train"].map(tokenize_function, batched=True)
tokenized_dataset1 = tk_dataset.map(lambda x: {"labels": pad_labels(x["labels"], max_length)}) '''


Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

TypeError: 'int' object is not iterable

In [172]:
for batch in tokenized_data["train"].take(3):
    print("Labels:", batch["labels"])

Labels: [4, 4, 4]
Labels: [4]
Labels: [4]


In [163]:
def verify_shapes(dataset, columns=["input_ids", "attention_mask", "labels"]):
    for split in dataset.keys():  # Iterate over splits (e.g., 'train', 'validation')
        print(f"Verifying shapes in split: {split}")
        for column in columns:
            # Get unique lengths for the column
            lengths = set(len(item) for item in dataset[split][column])
            if len(lengths) > 1:
                print(f"Inconsistent shapes found in {column} of {split}: {lengths}")
            else:
                print(f"{column} in {split} has consistent shape: {lengths}")

# Use the function to verify the shapes in your dataset
verify_shapes(tokenized_data, columns=["input_ids", "attention_mask", "labels"])



Verifying shapes in split: train
input_ids in train has consistent shape: {128}
attention_mask in train has consistent shape: {128}
Inconsistent shapes found in labels of train: {1, 66, 3, 36, 6, 136, 105, 10, 45, 78, 15, 21, 55, 120, 91, 28, 190}
Verifying shapes in split: validation
input_ids in validation has consistent shape: {128}
attention_mask in validation has consistent shape: {128}
Inconsistent shapes found in labels of validation: {1, 66, 3, 36, 6, 136, 105, 10, 171, 45, 78, 15, 21, 55, 153, 91, 28, 190}
Verifying shapes in split: test
input_ids in test has consistent shape: {128}
attention_mask in test has consistent shape: {128}
Inconsistent shapes found in labels of test: {1, 66, 3, 36, 6, 10, 171, 45, 78, 15, 465, 210, 21, 55, 120, 91, 28}


In [145]:
def manual_pad_labels(labels_column, max_length=128, pad_value=-100):
    padded_labels = []
    for seq in labels_column:
        # Flatten the list if it contains nested lists
        if any(isinstance(subseq, list) for subseq in seq):
            flat_seq = [item for subseq in seq for item in subseq]  # Flatten the nested list
        else:
            flat_seq = seq

        # Pad or truncate the flattened sequence
        padded_seq = flat_seq[:max_length] + [pad_value] * (max_length - len(flat_seq))
        padded_labels.append(padded_seq)
    return padded_labels


In [155]:

# Apply flattening and padding to the `labels` column
tokenized_dataset1 = tokenized_data.map(lambda x: {"labels": manual_pad_labels(x["labels"])}, batched=True)



Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [156]:
# Print padded labels to inspect
for i in range(5):
    print(f"Sample {i+1} labels:", tokenized_dataset1["train"]["labels"][i])


Sample 1 labels: [4, 4, 4, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100]
Sample 2 labels: [4, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100

In [157]:
# Use the function to verify the shapes in your dataset
verify_shapes(tokenized_dataset1, columns=["input_ids", "attention_mask", "labels"])

Verifying shapes in split: train
input_ids in train has consistent shape: {128}
attention_mask in train has consistent shape: {128}
labels in train has consistent shape: {128}
Verifying shapes in split: validation
input_ids in validation has consistent shape: {128}
attention_mask in validation has consistent shape: {128}
labels in validation has consistent shape: {128}
Verifying shapes in split: test
input_ids in test has consistent shape: {128}
attention_mask in test has consistent shape: {128}
labels in test has consistent shape: {128}


In [ ]:
tokenized_dataset1['train']['entity_pairs']

In [117]:
tokenized_dataset1

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'ner_tags_label', 'entity_pairs', 'marked_sentences', 'relations', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'ner_tags_label', 'entity_pairs', 'marked_sentences', 'relations', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'ner_tags_label', 'entity_pairs', 'marked_sentences', 'relations', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 3453
    })
})

In [173]:
def is_not_empty(example):
    """Checks if the 'entity_pairs' field is not empty."""
    return len(example["entity_pairs"]) > 0 and example["entity_pairs"][0] != ('', '', 'NO_RELATION')  # Check if not empty or default NO_RELATION

# Apply the filter
filtered_dataset = tokenized_data.filter(is_not_empty)

Filter:   0%|          | 0/14041 [00:00<?, ? examples/s]

Filter:   0%|          | 0/3250 [00:00<?, ? examples/s]

Filter:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [174]:
verify_shapes(filtered_dataset, columns=["input_ids", "attention_mask", "labels"])

Verifying shapes in split: train
input_ids in train has consistent shape: {128}
attention_mask in train has consistent shape: {128}
Inconsistent shapes found in labels of train: {1, 66, 3, 36, 6, 136, 105, 10, 45, 78, 15, 21, 55, 120, 91, 28, 190}
Verifying shapes in split: validation
input_ids in validation has consistent shape: {128}
attention_mask in validation has consistent shape: {128}
Inconsistent shapes found in labels of validation: {1, 66, 3, 36, 6, 136, 105, 10, 171, 45, 78, 15, 21, 55, 153, 91, 28, 190}
Verifying shapes in split: test
input_ids in test has consistent shape: {128}
attention_mask in test has consistent shape: {128}
Inconsistent shapes found in labels of test: {1, 66, 3, 36, 6, 10, 171, 45, 78, 15, 465, 210, 21, 55, 120, 91, 28}


In [112]:
''' # Convert tokenized dataset to TensorFlow format with the correct label shape
train_dataset = filtered_dataset["train"].to_tf_dataset(
    columns=["input_ids", "attention_mask"],
    label_cols=["labels"],
    shuffle=True,
    batch_size=8,
)
val_dataset = filtered_dataset["validation"].to_tf_dataset(
    columns=["input_ids", "attention_mask"],
    label_cols=["labels"],
    shuffle=False,
    batch_size=8,
) '''

/usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:403: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(


In [164]:
filtered_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'ner_tags_label', 'entity_pairs', 'marked_sentences', 'relations', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'ner_tags_label', 'entity_pairs', 'marked_sentences', 'relations', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'ner_tags_label', 'entity_pairs', 'marked_sentences', 'relations', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 3453
    })
})

In [159]:
import tensorflow as tf

def to_tf_dataset(tokenized_datasets):
    return tf.data.Dataset.from_tensor_slices((
        {
            "input_ids": tokenized_datasets["input_ids"],
            "attention_mask": tokenized_datasets["attention_mask"]
        },
        tokenized_datasets["labels"]
    )).shuffle(len(tokenized_datasets["input_ids"])).batch(8)

train_dataset = to_tf_dataset(filtered_dataset["train"])
val_dataset = to_tf_dataset(filtered_dataset["validation"])


In [165]:
def to_tf_dataset(tokenized_datasets):
        # Reshape labels to match logits shape
        labels = tf.reshape(tokenized_datasets["labels"], (-1, max_length))
        # Ensure that label padding is done correctly
        labels = tf.where(labels == -100, label2id["NO_RELATION"], labels)  # Ensure labels are int
        return tf.data.Dataset.from_tensor_slices((
            {
                "input_ids": tokenized_datasets["input_ids"],
                "attention_mask": tokenized_datasets["attention_mask"]
            },
            labels
        )).shuffle(len(tokenized_datasets["input_ids"])).batch(8)
train_data = to_tf_dataset(filtered_dataset["train"])
val_data = to_tf_dataset(filtered_dataset["validation"])

In [106]:
# Example check for `None` values in inputs
assert filtered_dataset["train"]["input_ids"] is not None, "input_ids cannot be None"
assert filtered_dataset["train"]["attention_mask"] is not None, "attention_mask cannot be None"
assert filtered_dataset["train"]["labels"] is not None, "labels cannot be None"


In [64]:
print("Shape of input_ids:", len(filtered_dataset["train"]["input_ids"][0]))
print("Shape of attention_mask:", len(filtered_dataset["train"]["attention_mask"][0]))
print("Shape of labels:", len(filtered_dataset["train"]["labels"][0]))


Shape of input_ids: 128
Shape of attention_mask: 128
Shape of labels: 128


In [73]:
filtered_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'ner_tags_label', 'entity_pairs', 'marked_sentences', 'relations', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 6136
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'ner_tags_label', 'entity_pairs', 'marked_sentences', 'relations', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1579
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'ner_tags_label', 'entity_pairs', 'marked_sentences', 'relations', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1483
    })
})

In [52]:
BATCH_SIZE=16
NUM_EPOCHS=2

In [107]:
batches_per_epoch = len(tokenized_dataset1["train"]) // BATCH_SIZE
total_train_steps = int(batches_per_epoch*NUM_EPOCHS)

In [234]:
from transformers import create_optimizer
#optimizer, schedule = create_optimizer(init_lr=2e-5,num_warmup_steps=0, num_train_steps=total_train_steps,)

In [197]:
from transformers import AdamW,create_optimizer,TFRobertaForSequenceClassification  # Import AdamW from transformers instead of Keras

   # Load model for sequence classification
model = TFRobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=len(label2id))
# Use create_optimizer to set up the optimizer
optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=total_train_steps
)
#model.compile(optimizer=optimizer,  metrics=["accuracy"])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

In [198]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy

model.compile(
    optimizer=optimizer,
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

In [109]:
for batch in train_dataset.take(1):
    input_ids, labels = batch
    print("Shape of input_ids:", input_ids["input_ids"].shape)
    print("Shape of labels:", labels.shape)


Shape of input_ids: (8, 128)
Shape of labels: (8, 128)


In [181]:
train_dataset = filtered_dataset["train"].to_tf_dataset(
    columns=["input_ids", "attention_mask"],
    label_cols=["labels"],
    shuffle=True,
    batch_size=8  # Set a fixed batch size
)

val_dataset = filtered_dataset["validation"].to_tf_dataset(
    columns=["input_ids", "attention_mask"],
    label_cols=["labels"],
    shuffle=False,
    batch_size=8  # Set a fixed batch size
)


/usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:403: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(


In [188]:
def tokenize_function(examples):
    # Tokenize the input sentences
    marked_sentences = [str(sentence) if sentence is not None else "" for sentence in examples["marked_sentences"]]
    encodings = tokenizer(
        marked_sentences,
        truncation=True,
        padding="max_length",
        max_length=128
    )

    # Flatten relations to ensure each is a single scalar label
    relations = examples["relations"]
    relations = [relation[0] if isinstance(relation, list) else relation for relation in relations]
    relations = [relation if relation else "NO_RELATION" for relation in relations]
    labels = [label2id.get(relation, label2id["NO_RELATION"]) for relation in relations]

    return {
        "input_ids": encodings["input_ids"],
        "attention_mask": encodings["attention_mask"],
        "labels": labels  # Ensuring labels are single scalar values
    }

# Apply the function to tokenize the dataset
tokenized_dataset = data.map(tokenize_function, batched=True)

# Convert to TensorFlow dataset with a fixed batch size
train_dataset = tokenized_dataset["train"].to_tf_dataset(
    columns=["input_ids", "attention_mask"],
    label_cols=["labels"],
    shuffle=True,
    batch_size=8  # Use a consistent batch size
)

val_dataset = tokenized_dataset["validation"].to_tf_dataset(
    columns=["input_ids", "attention_mask"],
    label_cols=["labels"],
    shuffle=False,
    batch_size=8
)


Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:403: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(


In [194]:
for batch in train_dataset.take(1):
    inputs, labels = batch  # Unpack the tuple into inputs and labels
    print("Batch labels shape:", labels.shape)  # Should print the shape of labels as (batch_size,)
    print("input ids shape:",inputs['input_ids'].shape)
    print("attention mask shape:",inputs['attention_mask'].shape)



Batch labels shape: (8,)
input ids shape: (8, 128)
attention mask shape: (8, 128)


In [199]:
model.fit(train_dataset, validation_data=val_dataset, epochs=2)



Epoch 1/2
1756/1756 [==============================] - 510s 266ms/step - loss: 0.2155 - accuracy: 0.9326 - val_loss: 0.0989 - val_accuracy: 0.9729
Epoch 2/2
1756/1756 [==============================] - 472s 269ms/step - loss: 0.0761 - accuracy: 0.9788 - val_loss: 0.0989 - val_accuracy: 0.9729


In [ ]:
tokenized_dataset['train']['relations']

In [206]:
from sklearn.metrics import classification_report
import numpy as np

# Define relationship labels (adjust as needed for your model)
relationship_labels = ["ORG-LOC", "PER-ORG", "PER-PER", "ORG-ORG", "NO_RELATION"]

# Get predictions
preds = model.predict(val_dataset)
pred_labels = np.argmax(preds.logits, axis=-1)  # Predicted class indices

# Get true labels directly from the dataset
true_labels = tokenized_dataset["validation"]["labels"]

# Map label indices to relationship names for true and predicted labels
true_label_names = [relationship_labels[label] for label in true_labels]
predicted_label_names = [relationship_labels[pred] for pred in pred_labels]

# Generate classification report
print(classification_report(true_label_names, predicted_label_names, target_names=relationship_labels))



407/407 [==============================] - 35s 86ms/step
              precision    recall  f1-score   support

     ORG-LOC       0.98      0.99      0.99      2758
     PER-ORG       0.96      0.67      0.79        75
     PER-PER       0.94      0.94      0.94       191
     ORG-ORG       0.86      0.77      0.81        56
 NO_RELATION       0.96      0.89      0.92       170

    accuracy                           0.97      3250
   macro avg       0.94      0.85      0.89      3250
weighted avg       0.97      0.97      0.97      3250



In [205]:
# Define relationship labels for the model
relationship_labels = ["ORG-LOC", "PER-ORG", "PER-PER", "ORG-ORG", "NO_RELATION"]

def predict_relationship(text, entity1, entity2):
    # Format the text with special entity markers for the model to recognize entity pairs
    marked_text = text.replace(entity1, f"[E1]{entity1}[/E1]").replace(entity2, f"[E2]{entity2}[/E2]")

    # Tokenize the text with entity markers
    inputs = tokenizer(marked_text, return_tensors="tf", padding="max_length", max_length=128, truncation=True)

    # Run the model to get predictions
    logits = model(inputs).logits
    prediction = tf.argmax(logits, axis=-1).numpy()[0]  # Get the predicted class index

    # Map the prediction to a relationship label
    relationship = relationship_labels[prediction]

    return f"Relationship between '{entity1}' and '{entity2}': {relationship}"

# Example usage
text = "Hugging Face Inc. is based in New York City."
entity1 = "Hugging Face Inc."
entity2 = "New York City"
print(predict_relationship(text, entity1, entity2))


Relationship between 'Hugging Face Inc.' and 'New York City': ORG-LOC


In [207]:
# Specify the directory where you want to save the model
output_dir = "/content/sample_data/relationship_extraction"

# Save the model using save_pretrained
model.save_pretrained(output_dir)

In [209]:
import json
import os

with open(os.path.join(output_dir, "label2id.json"), "w") as f:
    json.dump(label2id, f)

with open(os.path.join(output_dir, "id2label.json"), "w") as f:
    json.dump(id2label, f)

In [210]:
tokenizer.save_pretrained(output_dir)

('/content/sample_data/relationship_extraction/tokenizer_config.json',
 '/content/sample_data/relationship_extraction/special_tokens_map.json',
 '/content/sample_data/relationship_extraction/vocab.json',
 '/content/sample_data/relationship_extraction/merges.txt',
 '/content/sample_data/relationship_extraction/added_tokens.json',
 '/content/sample_data/relationship_extraction/tokenizer.json')